#### 1. create a dataframe

In [0]:
green_taxi = spark.read.csv("/Volumes/azuredatabricks/default/greentaxi/2021_Green_Taxi_Trip_Data.csv", header=True, inferSchema=True)


In [0]:
green_taxi_repartition = green_taxi.repartition(10)
green_taxi_repartition.write.format("parquet").mode('overwrite').save("/Volumes/azuredatabricks/default/read_stream/")

In [0]:
green_taxi.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, current_timestamp, window, col
from pyspark.sql.types import TimestampType


green_taxi_schema = green_taxi.schema

# Create ReadStream
green_taxi_read = spark.readStream \
    .format("parquet") \
    .schema(green_taxi_schema) \
    .option("header", "true") \
    .load("/Volumes/azuredatabricks/default/read_stream/") \
    .withColumn('timestamp', current_timestamp().cast(TimestampType()))

green_taxi_read = green_taxi_read.withWatermark("timestamp", "5 minutes")

taxiTransformation = green_taxi_read \
    .groupBy(
        col("timestamp"),
        col("trip_type")
    ) \
    .sum("fare_amount")

# WriteStream
taxiWriteStreams = taxiTransformation.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/Volumes/azuredatabricks/default/write_stream/") \
    .option("checkpointLocation", "/Volumes/azuredatabricks/default/checkpoint/") \
    .start()

# transformed_df = spark.readStream \
#     .format("parquet") \
#     .schema(taxiTransformation.schema) \
#     .option("header", "true") \
#     .load("/Volumes/azuredatabricks/default/write_stream/")

green_taxi_read.createOrReplaceTempView('newdf')

result_df = spark.sql("SELECT VendorID,timestamp, count(*)  FROM newdf WHERE trip_type = 1 GROUP BY  VendorID, timestamp")

write_2 = result_df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/Volumes/azuredatabricks/default/write_stream2/") \
    .option("checkpointLocation", "/Volumes/azuredatabricks/default/checkpoint2/") \
    .start()

write_2.awaitTermination()
taxiWriteStreams.awaitTermination()




com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio